In [4]:
import numpy as np
from sklearn.decomposition import PCA
import pylab as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import pickle
import os

In [5]:
# Se cargan los datos
y_total = np.genfromtxt("msd_genre_dataset.txt",dtype =None,usecols=0,comments=None,delimiter=',',skip_header=10)
x_total = np.genfromtxt("msd_genre_dataset.txt",comments=None,delimiter=',',skip_header=10)[:,4:]

generos = [b'metal',b'punk',b'dance and electronica']

y_raw = []
x_raw = []
for i in range(0,y_total.size):
    if y_total[i] in generos:
        y_raw.append(y_total[i])
        x_raw.append(x_total[i])
        
y_raw = np.asarray(y_raw)
x_raw = np.asarray(x_raw)

In [7]:
# Preprocesamiento
x_normalizado = (x_raw - np.mean(x_raw,axis=0))/np.std(x_raw,axis=0)

# PCA
proporcion_varianza = 0
n_componentes = 0
while proporcion_varianza < 0.90:
    n_componentes = n_componentes + 1
    pca = PCA(n_components=n_componentes)
    pca = pca.fit(x_normalizado)
    proporcion_varianza = np.sum(pca.explained_variance_ratio_)

suma_acumulada_varianza = np.cumsum(pca.explained_variance_ratio_*100)

x_preprocesamiento = pca.fit_transform(x_normalizado)
xy_preprocesamiento = np.append(x_preprocesamiento,np.array([y_raw]).T,axis=1)

# Separacion de datos para prueba y entrenamiento
proporcion_prueba = 0.1
np.random.shuffle(xy_preprocesamiento)
n_prueba = round(proporcion_prueba*y_raw.size)
n_entrenamiento = y_raw.size - n_prueba
datos_prueba = xy_preprocesamiento[0:n_prueba,:]
x_prueba = datos_prueba[:,0:-1].astype(np.float64)
y_prueba = datos_prueba[:,-1]
datos_entrenamiento = xy_preprocesamiento[n_prueba:,:]
x_entrenamiento = datos_entrenamiento[:,0:-1].astype(np.float64)
y_entrenamiento = datos_entrenamiento[:,-1]
etiquetas_prueba = np.where(y_prueba == generos[2], np.ones(n_prueba), -np.ones(n_prueba))
etiquetas_entrenamiento = np.where(y_entrenamiento == generos[2], np.ones(n_entrenamiento), -np.ones(n_entrenamiento))

In [4]:
# Generacion del clasificador
puntajes = []
clasificadores_2L_NN = []
promedio_coefs = []
n_n_min = 2
n_n_max = 1000
for n_neuronas in range(n_n_min,n_n_max+1):
    clasificador = MLPClassifier(solver='sgd',activation='relu',alpha=1e-5,
                                       hidden_layer_sizes=(n_neuronas,),batch_size=10,max_iter=10000)
    clasificadores_2L_NN.append(clasificador.fit(x_entrenamiento, etiquetas_entrenamiento))
    puntajes.append(clasificador.score(x_prueba_t, etiquetas_prueba_t))
    promedio_coefs.append(np.average(clasificador.coefs_[0],axis=1))

In [5]:
plt.plot(puntajes)
plt.savefig('test_4.pdf')
plt.close()

In [7]:
a = np.asarray(promedio_coefs)
for i in range(0,17):
    plt.plot(np.abs(a[:,i]))
plt.savefig('test_coef_4.pdf')
plt.close()

In [10]:
os.mkdir("saved")

In [12]:
pickle.dump(clasificadores_2L_NN, open("saved/clasificadores_2L_NN_1000.p", "wb"))
pickle.dump(puntajes, open("saved/puntajes_2L_NN_1000.p", "wb"))

In [5]:
# Generacion del clasificador
puntajes_log = []
clasificadores_2L_NN_log = []
n_n_min_2 = 2
n_n_max_2 = 200
for n_neuronas in range(n_n_min_2,n_n_max_2+1):
    clasificador = MLPClassifier(solver='sgd',activation='logistic',
                                       hidden_layer_sizes=(n_neuronas,),batch_size=100,max_iter=10000)
    clasificadores_2L_NN_log.append(clasificador.fit(x_entrenamiento, etiquetas_entrenamiento))
    puntajes_log.append(clasificador.score(x_prueba_t, etiquetas_prueba_t))    

In [6]:
pickle.dump(clasificadores_2L_NN_log, open("saved/clasificadores_2L_NN_log.p", "wb"))
pickle.dump(puntajes_log, open("saved/puntajes_2L_NN_log.p", "wb"))

In [7]:
plt.plot(puntajes_log)
plt.savefig('test_4_log.pdf')
plt.close()

In [8]:
# Generacion del clasificador
puntajes_log_2 = []
clasificadores_2L_NN_log_2 = []
for n_neuronas in range(n_n_min_2,n_n_max_2+1):
    clasificador = MLPClassifier(solver='sgd',activation='logistic',alpha=1e-4,
                                       hidden_layer_sizes=(n_neuronas,),batch_size=100,max_iter=10000)
    clasificadores_2L_NN_log_2.append(clasificador.fit(x_entrenamiento, etiquetas_entrenamiento))
    puntajes_log_2.append(clasificador.score(x_prueba_t, etiquetas_prueba_t))  

In [9]:
plt.plot(puntajes_log_2)
plt.savefig('test_4_log_2.pdf')
plt.close()

In [25]:
pickle.dump(clasificadores_2L_NN_log_2, open("saved/clasificadores_2L_NN_log_2.p", "wb"))
pickle.dump(puntajes_log_2, open("saved/puntajes_2L_NN_log_2.p", "wb"))

In [12]:
#clasificadores_2L_NN_relu = pickle.load(open("saved/clasificadores_2L_NN_1000.p", "rb"))
puntajes_2L_NN_relu = pickle.load(open("saved/puntajes_2L_NN_1000.p", "rb"))

In [34]:
n_neuronas_optimo_relu = np.argmax(np.asarray(puntajes_2L_NN_relu)) + n_n_min_2
n_neuronas_optimo_relu

396

In [32]:
x_entrenamiento_total = np.append(x_entrenamiento,x_prueba_t,axis=0)
y_entrenamiento_total = np.append(y_entrenamiento,y_prueba_t,axis=0)
etiquetas_entrenamiento_total = np.where(y_entrenamiento_total == generos[2],
                                         np.ones(n_entrenamiento+n_prueba_t), -np.ones(n_entrenamiento+n_prueba_t))
clasificador_escogido_relu = MLPClassifier(solver='sgd',activation='relu',
                                       hidden_layer_sizes=(n_neuronas_optimo_relu,),batch_size=100,max_iter=10000)
clasificador_escogido_relu.fit(x_entrenamiento_total, etiquetas_entrenamiento_total)
puntajes_c_escogido = clasificador_escogido_relu.score(x_prueba_f, etiquetas_prueba_f)

In [33]:
puntajes_c_escogido

0.89127604166666663

In [42]:
# Generacion del clasificador
puntajes = []
clasificadores_2L_NN = []
promedio_coefs = []
n_n_min = 2
n_n_max = 450
n = n_n_min
while n <= n_n_max:
    clasificador = MLPClassifier(solver='sgd',activation='relu',alpha=1e-5,
                                       hidden_layer_sizes=(n_neuronas,),batch_size=10,max_iter=10000)
    clasificadores_2L_NN.append(clasificador.fit(x_entrenamiento, etiquetas_entrenamiento))
    puntajes.append(clasificador.score(x_prueba_t, etiquetas_prueba_t))
    promedio_coefs.append(np.average(clasificador.coefs_[0],axis=1))
    n = n + 5

In [36]:
pickle.dump(clasificadores_2L_NN, open("saved/clasificadores_2L_NN_relu.p", "wb"))
pickle.dump(puntajes, open("saved/puntajes_2L_NN_relu.p", "wb"))

In [43]:
plt.plot(puntajes)
plt.savefig('test_4_relu_450_2.pdf')
plt.close()

In [39]:
n_neuronas_optimo_relu_2 = np.argmax(np.asarray(puntajes))*5 + n_n_min_2
n_neuronas_optimo_relu_2

97

In [14]:
# Generacion del clasificador
puntajes_matriz = []
n_n_min = 2
n_n_max = 102
n_l1 = n_n_min
while n_l1 <= n_n_max:
    n_l2 = n_n_min
    puntajes_l2 = []
    clasificadores_l2 = []
    while n_l2 <= n_n_max:
        clasificador = MLPClassifier(solver='lbfgs',activation='relu',hidden_layer_sizes=(n_l1,n_l2))
        clasificador = clasificador.fit(x_entrenamiento, etiquetas_entrenamiento)
        puntajes_l2.append(clasificador.score(x_prueba, etiquetas_prueba))
        n_l2 = n_l2 + 10
    puntajes_matriz.append(puntajes_l2)
    n_l1 = n_l1 + 10

In [15]:
plt.imshow(np.rot90(1-np.asarray(puntajes_matriz).T,1),cmap=plt.cm.hot_r,
           extent=[n_n_min,n_n_max,n_n_min,n_n_max])
plt.colorbar()
plt.xlabel('Neuronas en la segunda capa escondida')
plt.ylabel('Neuronas en la primera capa escondida')
plt.savefig('puntajes_matriz_2.png',dpi=300)
plt.close()

In [16]:
pickle.dump(puntajes_matriz, open("saved/puntajes_matriz_2.p", "wb"))

In [ ]:
# Generacion del clasificador
puntajes_matriz_2 = []
n_n_min = 2
n_n_max = 16
n_n_max_2 = 102
n_c1 = n_n_min
while n_c1 <= n_n_max:
    n_c2 = n_n_min
    puntajes_c2 = []
    while n_l2 <= n_n_max_2:
        clasificador = MLPClassifier(solver='lbfgs',activation='relu',hidden_layer_sizes=(n_l1,n_l2))
        puntajes = cross_val_score(clasificador, x_entrenamiento, etiquetas_entrenamiento, cv = 4, n_jobs = 2)
        puntajes_c2.append(puntajes)
        n_c2 = n_c2 + 4
    puntajes_matriz_2.append(puntajes_c2)
    n_c1 = n_c1 + 4